<h1>Quick context :</h1> 

I'm trying to build the model of a small Spacecraft using SysML2.
<br>
My reference for doing that is the Powerpoint presentation of August 21: <u>Intro to Pilot SysML v2 Language</u>. I encountered some issues, so my questions will be guided from there.
<br>
<br>
Here are some references to the SysML 1 model :



<hr> 

![containment](https://imgur.com/drgqZzK.jpg) 

![bdd](https://imgur.com/iKteipO.jpg)

![ibd_mission](https://imgur.com/wsKMYn5.jpg)


<hr>

# Going into SysML2

### Recreating the structure :

In [15]:
package 'Structure'{
    package 'Interfaces'{}
    block 'Mission';
    block 'Ground Station';
    block 'Spacecraft Vehicle';
}

Package Structure (3a301f08)


### Value properties, ports or interface definitions (no apparent issues)

In [13]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value inout data : Real; //flow properties => inout
            //no references ?
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;
        }
    }
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
        }
        
        interface : CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
    }
      
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
        
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
}

Package Structure (b0c1ad0)


 <hr>

## Defining activities and actions

This is where the troubles begin.<br>
<span style="color:red">For the sake of clarity, I'll just expose the blocks that are useful, the cells won't be runnable anymore, but the full version is available down below.</span><br>
I listed the activities each block contains and tried to depict and formalize the flow of actions happening in said activities.</br>

In [ ]:
    block Data;
    block Signal; //I did not find any other solutions to have signals flowing from one activity to another
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
        
        
        activity PingSpacecraft(out ping:Signal){
            // 1. send signal on CommunicationLink interface 
        } 
        
        activity ReceiveData(in dataPacket:Data){
            // 0. merge node
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
        }
    }
    
    
    
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
       
        activity ConsumePower(in deviceOff:Signal, out batteryLow:Signal){
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
        }

        activity RechargeBatteries(out fullCharge:Signal){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1.
        }
        
        activity SendDataFrames(out dataPacket:Data){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
        }
        
        
        activity TransmitData(in batteryLow:Signal, 
                              out deviceOff:Signal, 
                              out transmissionDone:Signal){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done  
    }

  <hr>

<b style="color:red">I encountered many issues when trying to actually implement those activities, and I'll now try to go over them one by one.</b>

<h2>Operations on value properties</h2><br>
Is it possible to make operations on the value properties of a block in an action? <br>
An example of a use-case would be on line 1. of SendDataFrames activity:

In [ ]:
activity SendDataFrames(out dataPacket:Data){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
        }

<hr>

<h2>Forks and action flow</h2>
<ul>
    <li>Are forks a thing in SysML2, or is it simply not specifying a 'then' before calling the following action?</li>
    <li>What happens if I call multiple actions in an activity, without calling one 'then' the other? Will they run simultaneously?</li>
    <li>Is the keyword 'then' the best way to represent an action flow inside an activity?</li>
</ul>

<hr>

<h2>Merge and decision nodes</h2>
The following example will be useful:

![act_recharge](https://imgur.com/fKHl12u.jpg)
I tried defining the merge node following the example slide 31:

In [ ]:
activity RechargeBatteries(){
    first start;
    then[0..1] action mergeNode : Activities::Merge;
    // decision node
        // battery++;
        then[0..1] mergeNode;
    //else ...
    }

<h4>Here are some questions:</h4>
<ul>
    <li>Is the merge node correctly defined in this case?</li>
    <li>What is the multiplicity ( [0..1] ) used for? What does it define?</li>
    <li>Is it possible to use Activities::Decision to define a Decision Node, and if so: How?

<hr>

<h2>Sending signals to other parts/activities</h2>
Using the simplest example :

![send_ping](https://imgur.com/GX0t4TL.jpg)

I want the ping to be sent to the Spacecraft Vehicle via the CommunicationInterface on port groundCommIF.<br>
<ul>
    <li>How would one define this activity? (See right down below)</li>
    <li>Is the target "Spacecraft Vehicle", groundPort, spacePort or CommunicationLink?</li>
    <li>Do I need to define a block Signal, or is there an intergrated thing for this kind of signaling?</li>
</ul>

In [ ]:
send ping:Signal to "Spacecraft Vehicle";

^ This line prompted an error requiring an "of" argument.

In [ ]:
send ping:Signal of null to "Spacecraft Vehicle";

^ This ran without error, but what is the null expected to be? This is not referred to in the slides

 <hr>

<h2>Receiving a signal from another part/activity</h2>
<br>
Using the receiving end from the last example, where would I define the trigger? In the SysML1 model, WaitingforGroundStationPing is a state in the state-machine diagram, but never is there an activity that awaits the trigger of the ping.
<br><br>
Using another example:

![receiving_ping](https://imgur.com/ZGEanDG.jpg)

Following slide 30 (Signaling), I would be tempted to write the following:

In [ ]:
activity ReceiveData(in data:Data flow from "Spacecraft Vehicle"::SendDataFrames::data){
    action merge : Activities::Merge;
    accept trigger(data:Data);
    then action processData(data);
    then[0..1] merge;
}

But I'm really not sure I understand how the (accept) trigger works. 
<ul><li>How does the trigger know where to "sniff" for data? Should I pass as argument the port on which I want the trigger to listen instead?</li>
    <li>If not, some indications would be appreciated</li></ul>
        
But the "Spacecraft Vehicle" block is not accessible from this function which I understand, and that bring me to my next question:


<h2>Arguments flowing between activities</h2>
<ul>
    <li>How does one activity accept "in arguments" from activities nested in another block?</li> 
    <li>Do I need to define the activities with 'action' from the Mission block?</li>
</ul>

From my (basic) understanding, the SysML1 model has both the actions and activities defined inside the GroundStation and Spacecraft blocks. And they’re called once the blocks are "instantiated" (part properties) in the Mission block.<br>
But it seems to me like the Action Allocation slide (s. 32) suggests that I should call the activities myself while instantiating the blocks.

### So here is what I have for now, I'm blocked at the activity definition

In [2]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value inout data : Real; //flow properties => inout
            //no references ?
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;
        }
    }
    
    
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
        }
        
        interface : CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    
    
      block Data;
      block Signal; //I did not find any other solutions to have signals flowing from one activity to another
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
        
        
        activity PingSpacecraft(out ping:Signal){
            // 1. send signal on CommunicationLink interface 
        } 
        
        activity ReceiveData(in dataPacket:Data){
            // 0. merge node
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
        }
    }
    
    
    
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
       
        activity ConsumePower(in deviceOff:Signal, out batteryLow:Signal){
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
        }

        activity RechargeBatteries(out fullCharge:Signal){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1.
        }
        
        activity SendDataFrames(out dataPacket:Data){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
        }
        
        
        activity TransmitData(in batteryLow:Signal, 
                              out deviceOff:Signal, 
                              out transmissionDone:Signal){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done  
        }
        
                
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
}

Package Structure (e2ea939)


In [3]:
%show Structure::"Ground Station"

Block Ground Station (3fd19188)
  [FeatureMembership] PortUsage CommIF (79670fb8)
    [FeatureTyping] PortDefinition CommunicationInterface (4d1d80de)
    [Subsetting] PortUsage ports (3c5dbdf8)
  [Membership] Activity PingSpacecraft (652fc517)
    [Superclassing] Activity Action (23b8d9f3)
  [Membership] Activity ReceiveData (65769740)
    [ParameterMembership] Parameter data (784db959)
      [FeatureTyping] Block Data (549871b2)
      [Subsetting] Feature objects (1f387978)
    [Superclassing] Activity Action (23b8d9f3)
  [FeatureMembership] Action pingSpacecraft (40807d1b)
    [FeatureTyping] Activity PingSpacecraft (652fc517)
    [ParameterMembership] Parameter (6e961cd4)
      [Subsetting] Feature things (453d496b)
    [Subsetting] Action performedActions (2f5ac102)
  [FeatureMembership] Action receiveData (6c77c9b7)
    [FeatureTyping] Activity ReceiveData (65769740)
    [ParameterMembership] Parameter (66c4d917)
      [Redefinition] Parameter data (784db959)
    [FeatureMembersh

In [18]:
%show Structure::"Spacecraft Vehicle"

Block Spacecraft Vehicle (13021680)
  [FeatureMembership] PortUsage CommIF1 (705b2998)
    [FeatureTyping] PortDefinition CommunicationInterface (70750bfa)
    [Subsetting] PortUsage ports (7a0e7ecd)
  [Membership] Activity ConsumePower (ebd959c)
    [Superclassing] Activity Action (1e58512c)
  [Membership] Activity RechargeBatteries (479e4d69)
    [Superclassing] Activity Action (1e58512c)
  [Membership] Activity SendDataFrames (6bdac984)
    [ParameterMembership] Parameter data (62f446e5)
      [FeatureTyping] Block Data (64054b3c)
      [Subsetting] Feature objects (2116b68b)
    [Superclassing] Activity Action (1e58512c)
  [Membership] Activity TransmitData (697596ec)
    [Superclassing] Activity Action (1e58512c)
  [FeatureMembership] Action transmitData (5098a17d)
    [FeatureTyping] Activity TransmitData (697596ec)
    [ParameterMembership] Parameter data (12bd2230)
      [FeatureTyping] Block Data (64054b3c)
      [Subsetting] Feature objects (2116b68b)
    [Subsetting] Action 

In [9]:
%publish Structure

Saving element Package@fa124ff (Structure)
... element id is a2bbfe60-d329-11e9-9cc7-618afdf1f226
Saving element Package@1945113f (ScalarValues)
... element id is a2bf80d0-d329-11e9-9cc7-618afdf1f226
Saving element Package@3be4f71 (Base)
... element id is a2c2b520-d329-11e9-9cc7-618afdf1f226
Saving element Comment@47e4d9d0
... element id is a2c685b0-d329-11e9-9cc7-618afdf1f226
Saving relationship Annotation@1948ea69
... relationship id is a2cb1990-d329-11e9-9cc7-618afdf1f226
Saving element Classifier@3015db78 (Anything)
... element id is a2ce74f0-d329-11e9-9cc7-618afdf1f226
Saving relationship Superclassing@71f96dfb
... relationship id is a2d1a940-d329-11e9-9cc7-618afdf1f226
Saving relationship Membership@5c645b43
... relationship id is a2d4dd90-d329-11e9-9cc7-618afdf1f226
Saving element Comment@6ed06f69
... element id is a2d838f0-d329-11e9-9cc7-618afdf1f226
Saving relationship Annotation@49798e84
... relationship id is a2db6d40-d329-11e9-9cc7-618afdf1f226
Saving element DataType@39109

... element id is a4c84920-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@290b2086
... relationship id is a4cb7d70-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@597b9a5e
... relationship id is a4ceb1c0-d329-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@6a0659ac
... relationship id is a4d1e610-d329-11e9-9cc7-618afdf1f226
Saving element Step@69da0b12 (performances)
... element id is a4d54170-d329-11e9-9cc7-618afdf1f226
Saving relationship FeatureTyping@2764c546
... relationship id is a4d89cd0-d329-11e9-9cc7-618afdf1f226
Saving element LiteralUnbounded@79b08632
... element id is a4dbd120-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@4a37921d
... relationship id is a4df0570-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@d4a01c2 ($result)
... element id is a4e212b0-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@7d350ffd
... relationship id is a4e63160-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnPa

... relationship id is a6e13e10-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@915837d ($result)
... element id is a6e47260-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@366d9830
... relationship id is a6e7a6b0-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@22241b6c
... relationship id is a6eadb00-d329-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@2f5b8250
... relationship id is a6ee0f50-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@1a49beb5
... relationship id is a6f143a0-d329-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@f1d0004
... relationship id is a6f4ed20-d329-11e9-9cc7-618afdf1f226
Saving relationship EndFeatureMembership@56637cff
... relationship id is a6f82170-d329-11e9-9cc7-618afdf1f226
Saving element Comment@127a7272
... element id is a6fb7cd0-d329-11e9-9cc7-618afdf1f226
Saving relationship Annotation@2938127d
... relationship id is a6fed830-d329-11e9-9cc7-618afdf1f226
Saving element Featu

Saving relationship ReturnParameterMembership@5a2f9e9f
... relationship id is a8f85e40-d329-11e9-9cc7-618afdf1f226
Saving element LiteralInteger@de77232
... element id is a8fb9290-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@696c60dd
... relationship id is a8feedf0-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@4b46f2f2 ($result)
... element id is a9024950-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@5e8428a5
... relationship id is a9057da0-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@4f11c009
... relationship id is a908b1f0-d329-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@20749d9
... relationship id is a90cf7b0-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@5464c120
... relationship id is a9102c00-d329-11e9-9cc7-618afdf1f226
Saving relationship EndFeatureMembership@6a0cbc6f
... relationship id is a913d580-d329-11e9-9cc7-618afdf1f226
Saving relationship Superclassing@42d61375
... relationship id 

... relationship id is ab085280-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@20ee7aa ($result)
... element id is ab0b86d0-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@7424c59e
... relationship id is ab0ebb20-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@6947344a
... relationship id is ab139d20-d329-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@26d41711
... relationship id is ab171f90-d329-11e9-9cc7-618afdf1f226
Saving element Feature@37b52340
... element id is ab1a7af0-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@663bb8ef
... relationship id is ab1daf40-d329-11e9-9cc7-618afdf1f226
Saving element LiteralUnbounded@7e5843db
... element id is ab210aa0-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@1aaddc03
... relationship id is ab246600-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@3fdec336 ($result)
... element id is ab280f80-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@6d84f966

...

... relationship id is b573dd20-d329-11e9-9cc7-618afdf1f226
Saving element LiteralInteger@66c83fc8
... element id is b5773880-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@61e12078
... relationship id is b57a45c0-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@6d711df1 ($result)
... element id is b57da120-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@4469b1b8
... relationship id is b580d570-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@5396a7aa
... relationship id is b58409c0-d329-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@60d1b21f
... relationship id is b587da50-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@2c63abad
... relationship id is b58c4720-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@4aaef27c
... relationship id is b58f7b70-d329-11e9-9cc7-618afdf1f226
Saving relationship EndFeatureMembership@4e2916c3
... relationship id is b592afc0-d329-11e9-9cc7-618afdf1f226
Saving element

Saving relationship Subsetting@2247efcf
... relationship id is b7881720-d329-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@458ac0d2
... relationship id is b78b4b70-d329-11e9-9cc7-618afdf1f226
Saving relationship ParameterMembership@57d0fc89
... relationship id is b78e7fc0-d329-11e9-9cc7-618afdf1f226
Saving element OperatorExpression@3fa76c61
... element id is b791db20-d329-11e9-9cc7-618afdf1f226
Saving element FeatureReferenceExpression@1cbb3d3b
... element id is b795abb0-d329-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@59546cfe
... relationship id is b798e000-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@31264ea8
... relationship id is b79c1450-d329-11e9-9cc7-618afdf1f226
Saving element Parameter@849b99f ($result)
... element id is b79f6fb0-d329-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@62bb61e2
... relationship id is b7a27cf0-d329-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@6917ddaa
... relationship id is b7

... relationship id is b98d5d00-d329-11e9-9cc7-618afdf1f226
Saving element Comment@6818d900
... element id is b9906a40-d329-11e9-9cc7-618afdf1f226
Saving relationship Annotation@3350ebdd
... relationship id is b9939e90-d329-11e9-9cc7-618afdf1f226
Saving relationship Membership@3c0036b
... relationship id is b996abd0-d329-11e9-9cc7-618afdf1f226
Saving element Comment@1d25c1c
... element id is b999e020-d329-11e9-9cc7-618afdf1f226
Saving relationship Annotation@629ae7e
... relationship id is b99ced60-d329-11e9-9cc7-618afdf1f226
Saving relationship Membership@17814b1c
... relationship id is b9a021b0-d329-11e9-9cc7-618afdf1f226
Saving element Comment@5c0f79f0
... element id is b9a35600-d329-11e9-9cc7-618afdf1f226
Saving relationship Annotation@34a0ef00
... relationship id is b9a6b160-d329-11e9-9cc7-618afdf1f226
Saving relationship Membership@7ac9af2a
... relationship id is b9a9e5b0-d329-11e9-9cc7-618afdf1f226
Saving element Comment@9d1a267
... element id is b9acf2f0-d329-11e9-9cc7-618afdf1f

Saving relationship Superclassing@17aa8a11
... relationship id is bb9f9b30-d329-11e9-9cc7-618afdf1f226
Saving relationship Membership@5377414a
... relationship id is bba2a870-d329-11e9-9cc7-618afdf1f226
Saving relationship Import@73718374
... relationship id is bba5dcc0-d329-11e9-9cc7-618afdf1f226
Saving element Package@775d487c (Interfaces)
... element id is bba91110-d329-11e9-9cc7-618afdf1f226
Saving element PortDefinition@27ac16f6 (CommunicationInterface)
... element id is bbac9380-d329-11e9-9cc7-618afdf1f226
Saving element ValueProperty@1754e311 (data)
... element id is bbb015f0-d329-11e9-9cc7-618afdf1f226
Saving relationship FeatureTyping@65fb3e40
... relationship id is bbb32330-d329-11e9-9cc7-618afdf1f226
Saving element ValueProperty@59072e9d (values)
... element id is bbb65780-d329-11e9-9cc7-618afdf1f226
Saving element ValueType@3e33d73e (Value)
... element id is bbb98bd0-d329-11e9-9cc7-618afdf1f226
Saving relationship Superclassing@6981f8f3
... relationship id is bbbe94e0-d329-

java.util.ConcurrentModificationException
	at org.eclipse.emf.common.util.AbstractEList$EIterator.checkModCount(AbstractEList.java:751)
	at org.eclipse.emf.common.util.AbstractEList$EIterator.doNext(AbstractEList.java:699)
	at org.eclipse.emf.common.util.AbstractEList$EIterator.next(AbstractEList.java:685)
	at org.eclipse.uml2.common.util.DerivedEObjectEList$DerivedListIterator.scanNext(DerivedEObjectEList.java:70)
	at org.eclipse.uml2.common.util.DerivedEObjectEList$DerivedListIterator.prepareNext(DerivedEObjectEList.java:96)
	at org.eclipse.uml2.common.util.DerivedEObjectEList$DerivedListIterator.hasNext(DerivedEObjectEList.java:152)
	at org.eclipse.uml2.common.util.DerivedEObjectEList$DerivedListIterator.next(DerivedEObjectEList.java:161)
	at java.base/java.util.Iterator.forEachRemaining(Iterator.java:133)
	at java.base/java.util.Spliterators$IteratorSpliterator.forEachRemaining(Spliterators.java:1801)
	at java.base/java.util.stream.ReferencePipeline$Head.forEach(ReferencePipeline.j